# AtliQ Products Analysis

## Contents

1. [Introduction](#introduction)
2. [Data loading and preprocessing](#data-loading-and-preprocessing)
    - [Import libraries](#libraries)
    - [Load/Download database](#loaddownload-database)
    - [Helper functions](#helper-functions)
    - [Basic tables info](#basic-tables-info)
    - [Missing values](#missing-values)
    - [Duplicates](#duplicates)
    - [Other](#other)
3. [Analysis](#analysis)
    1. [Finding the bestsellers](#finding-the-bestsellers)
    2. [Popularity across time and markets](#popularity-across-time-and-markets)
    3. [Variant sales](#variant-sales)
    4. [Channel sales](#channel-sales)
    5. [Product margin](#product-margins)
    6. [Price vs cost]
4. [Conclusion]

---

## Introduction

Our team has been commissioned by AtliQ Hardware to conduct a thorough analysis of their product portfolio and sales data.

As a prominent computer hardware producer in India, AtliQ is keen on enhancing their understanding of product performance. This analysis aims to identify top-selling products, uncover trends, and develop strategies to optimize sales and market share.

The primary objective of this research is to analyze AtliQ Hardware's product portfolio and sales data to better understand product performance and identify strategies for optimizing sales. We aim to answer these key questions:
- Which items are the bestsellers?
- How has popularity changed over time/across markets?
- Are there some variants that contribute a disproportionate amount to the product sales?
- Are some channels responsible for a large portion of a division’s sales?
- What are the products with the best/worst margin?
- Is gross price keeping up with manufacturing costs?

Through this analysis, our goal is to provide AtliQ Hardware with actionable insights and recommendations to help drive business growth.

[Back to Contents](#contents)

---

## Data loading and Preprocessing

### Libraries
These are the libraries that we are going to use for this project:

In [2]:
import pandas as pd
import sqlite3
import os
import requests
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

[Back to Contents](#contents)

---

### Load/Download database

We have access to an SQLite database with data on products, clients and sales. 

First let's check that it exists, and if doesn't, we'll download it.

In [3]:
# Local path to the Database
db_directory_path = 'Data'
db_file_path = os.path.join(db_directory_path, 'atliq_db.sqlite3')


In [4]:
# Check if directory exists. If it doesn't, create it
if not os.path.exists(db_directory_path):
    os.makedirs(db_directory_path)


In [5]:
# Check if file exists. If it doesn't, download it
if not os.path.exists(db_file_path):
    print('Database not found. Downloading the file...')

    db_url = 'https://practicum-content.s3.us-west-1.amazonaws.com/data-eng/databases/atliq_db.sqlite3'
    
    response = requests.get(db_url)
    with open(db_file_path, 'wb') as f:
        f.write(response.content)
    
    print('Database downloaded successfully!')
else:
    print('Database found.')


Database found.


We have our database. But we don't want to directly change the raw data, and we will be working directly with the database as much as possible. So we'll make a copy and modify that instead.

In [6]:
# Check if the copy exists
work_db_path = os.path.join(db_directory_path, 'atliq_db_processed.sqlite3')

if os.path.exists(work_db_path):
    print('Previous copy found.')
else:
    shutil.copyfile(db_file_path, work_db_path)
    print('Database duplicated.')


Previous copy found.


We can now connect to our working copy and start processing it. If we found that the copy already exists, we can assume that it is already processed, and we can skip those steps.

In [7]:
# Connect to the DB
con = sqlite3.connect(work_db_path)

Let's check that we have access to the tables that we are supposed to.

In [8]:
# Check all tables
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(*cursor.fetchall(), sep='\n')

('dim_customer',)
('dim_product',)
('fact_pre_discount',)
('fact_manufacturing_cost',)
('fact_gross_price',)
('fact_sales_monthly',)


[Back to contents](#contents)

---

### Helper functions

Lets check for missing values. We can't load the whole tables into pandas, so we'll have to rely only on SQL queries.

Lets build a function to help us, similar to pandas info().

We want to display the following:
- Table name
- Row count
- Column info, including name, type, null count, and Primary Key status.

In [9]:
# Count rows in the table
def row_count(table: str):
    query = f"""
    SELECT COUNT(*)
    from {table}
    """

    cursor.execute(query)
    return cursor.fetchone()[0]

In [10]:
# Find Null values in a column
def count_nulls_in_column(column: str, table: str):
    query = f"""
    SELECT COUNT(*)
    FROM {table}
    WHERE {column} IS NULL
    """

    cursor.execute(query)
    return cursor.fetchone()[0]

In [11]:
# Get all the column names from a table
def get_column_names(table: str):
    query = f"""
    PRAGMA table_info({table}) 
    """

    cursor.execute(query)
    result = cursor.fetchall()
    name_pos_in_row = 1

    column_names = []
    for row in result:
        column_names.append(row[name_pos_in_row])

    return column_names

In [12]:
# Check missing values in all columns of the table
def check_nulls(table: str):
    column_names = get_column_names(table)
    null_counts = []
    for column in column_names:
        null_counts.append((column, count_nulls_in_column(column, table)))

    return null_counts

In [13]:
def table_schema(table: str):
    query = f"""
    PRAGMA table_info({table})
    """

    return pd.read_sql_query(query, con)[['name', 'type', 'pk']]


In [14]:
def get_table_info(table: str):
    print(f'Table: {table}')
    print(f'Rows: {row_count(table)}')
    null_count = check_nulls(table)
    schema = table_schema(table)
    schema['nulls'] = [t[1] for t in null_count]
    display(schema)

To find duplicates, we'll compare the row count of a regular `SELECT` vs a `SELECT DISTINCT`, using relevant columns for each table.

In [15]:
def count_duplicates(table: str, list_cols: list, verbose=True):

    columns = ', '.join(list_cols)

    query = f"""
    SELECT COUNT(*)
    FROM {table}
    """
    cursor.execute(query)
    sel = cursor.fetchone()[0]

    query = f"""
    SELECT DISTINCT COUNT(*)
    FROM (
        SELECT DISTINCT {columns}
        FROM {table}
    )
    """
    cursor.execute(query)
    dis = cursor.fetchone()[0]
    duplicate_count = sel - dis

    if verbose:
        print(f'Table: {table}')
        print(f'Duplicates found: {duplicate_count}')
    return(duplicate_count)


[Back to Contents](#contents)

---

### Basic tables info

With our function ready, lets check the info on our tables.

In [16]:
display(get_table_info('dim_customer'))
cursor.execute("SELECT * FROM dim_customer LIMIT 5")
print(*cursor.fetchall(), sep='\n')

Table: dim_customer
Rows: 209


,name,type,pk,nulls
0,customer_code,INTEGER,0,0
1,customer,TEXT,0,0
2,platform,TEXT,0,0
3,channel,TEXT,0,0
4,market,TEXT,0,0
5,sub_zone,TEXT,0,0
6,region,TEXT,0,0


None

(70002017, 'Atliq Exclusive', 'Brick & Mortar', 'Direct', 'India', 'India', 'APAC')
(70002018, 'Atliq e Store', 'E-Commerce', 'Direct', 'India', 'India', 'APAC')
(70003181, 'Atliq Exclusive', 'Brick & Mortar', 'Direct', 'Indonesia', 'ROA', 'APAC')
(70003182, 'Atliq e Store', 'E-Commerce', 'Direct', 'Indonesia', 'ROA', 'APAC')
(70004069, 'Atliq Exclusive', 'Brick & Mortar', 'Direct', 'Japan', 'ROA', 'APAC')


In [17]:
get_table_info('dim_product')
cursor.execute("SELECT * FROM dim_product LIMIT 5")
print(*cursor.fetchall(), sep='\n')

Table: dim_product
Rows: 397


,name,type,pk,nulls
0,product_code,TEXT,0,0
1,division,TEXT,0,0
2,segment,TEXT,0,0
3,category,TEXT,0,0
4,product,TEXT,0,0
5,variant,TEXT,0,0
6,display_name,VARCHAR(20),0,379


('A0118150101', 'P & A', 'Peripherals', 'Internal HDD', 'AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM 256 MB Cache', 'Standard', 'AQ Dracula HDD')
('A0118150102', 'P & A', 'Peripherals', 'Internal HDD', 'AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM 256 MB Cache', 'Plus', 'AQ Dracula HDD')
('A0118150103', 'P & A', 'Peripherals', 'Internal HDD', 'AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM 256 MB Cache', 'Premium', 'AQ Dracula HDD')
('A0118150104', 'P & A', 'Peripherals', 'Internal HDD', 'AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM 256 MB Cache', 'Premium Plus', 'AQ Dracula HDD')
('A0219150201', 'P & A', 'Peripherals', 'Internal HDD', 'AQ WereWolf NAS Internal Hard Drive HDD – 8.89 cm', 'Standard', 'AQ Werewolf NAS HDD')


In [18]:
get_table_info('fact_pre_discount')
cursor.execute("SELECT * FROM fact_pre_discount LIMIT 5")
print(*cursor.fetchall(), sep='\n')

Table: fact_pre_discount
Rows: 1045


,name,type,pk,nulls
0,customer_code,INTEGER,0,0
1,fiscal_year,INTEGER,0,0
2,pre_invoice_discount_pct,float,0,0


(70002017, 2018, 0.0824)
(70002017, 2019, 0.0777)
(70002017, 2020, 0.0735)
(70002017, 2021, 0.0703)
(70002017, 2022, 0.1057)


In [19]:
get_table_info('fact_manufacturing_cost')
cursor.execute("SELECT * FROM fact_manufacturing_cost LIMIT 5")
print(*cursor.fetchall(), sep='\n')

Table: fact_manufacturing_cost
Rows: 1182


,name,type,pk,nulls
0,product_code,TEXT,0,0
1,cost_year,INTEGER,0,0
2,manufacturing_cost,float,0,0


('A0118150101', 2018, 4.619)
('A0118150101', 2019, 4.2033)
('A0118150101', 2020, 5.0207)
('A0118150101', 2021, 5.5172)
('A0118150102', 2018, 5.6036)


In [20]:
get_table_info('fact_gross_price')
cursor.execute("SELECT * FROM fact_gross_price LIMIT 5")
print(*cursor.fetchall(), sep='\n')

Table: fact_gross_price
Rows: 1182


,name,type,pk,nulls
0,product_code,TEXT,0,0
1,fiscal_year,INTEGER,0,0
2,gross_price,float,0,0


('A0118150101', 2018, 15.3952)
('A0118150101', 2019, 14.4392)
('A0118150101', 2020, 16.2323)
('A0118150101', 2021, 19.0573)
('A0118150102', 2018, 19.5875)


In [21]:
get_table_info('fact_sales_monthly')
cursor.execute("SELECT * FROM fact_sales_monthly LIMIT 5")
print(*cursor.fetchall(), sep='\n')

Table: fact_sales_monthly
Rows: 67178


,name,type,pk,nulls
0,date,TEXT,0,0
1,product_code,TEXT,0,0
2,customer_code,INTEGER,0,0
3,sold_quantity,INTEGER,0,0
4,fiscal_year,INTEGER,0,0


('2017-09-01', 'A0118150101', 70002017, 51, 2018)
('2017-09-01', 'A0118150101', 70002018, 77, 2018)
('2017-09-01', 'A0118150101', 70003181, 17, 2018)
('2017-09-01', 'A0118150101', 70003182, 6, 2018)
('2017-09-01', 'A0118150101', 70006157, 5, 2018)


There is only one missing value in the whole database. 

Curiously, none of the tables have primary keys declared. 

The product names are too long. Let's create shorter display names.

In [22]:
# Checking which products are used
query = """
SELECT DISTINCT product
FROM dim_product JOIN fact_sales_monthly USING (product_code)
"""

cursor.execute(query)
print(*cursor.fetchall(), sep='\n')

('AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM 256 MB Cache',)
('AQ WereWolf NAS Internal Hard Drive HDD – 8.89 cm',)
('AQ Zion Saga',)
('AQ Mforce Gen X',)


We only have sales data on this 4 products. We can ignore the rest.

In [23]:
# Define display names
display_names = [
    'AQ Dracula HDD', 
    'AQ Werewolf NAS HDD',
    'AQ Zion Saga',
    'AQ Mforce Gen X'
]

In [24]:
# Add the new column
query = """
ALTER TABLE dim_product
ADD display_name VARCHAR(20)
"""

try:
    cursor.execute(query);
except sqlite3.OperationalError:
    print('The column already exists.')


The column already exists.


In [25]:
# Update the table with the display names
# We match the left part of the display names with the product names

for name in display_names:
    query = f"""
    UPDATE dim_product
    SET display_name = '{name}'
    WHERE SUBSTR(product, 1, 15) LIKE SUBSTR('{name}', 1, 15) || '%'
    """

    cursor.execute(query)


In [26]:
# Check that the display names were correctly applied
query = """
SELECT DISTINCT display_name, product
FROM dim_product JOIN fact_sales_monthly USING (product_code)
"""

cursor.execute(query)
print(*cursor.fetchall(), sep='\n')

('AQ Dracula HDD', 'AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM 256 MB Cache')
('AQ Werewolf NAS HDD', 'AQ WereWolf NAS Internal Hard Drive HDD – 8.89 cm')
('AQ Zion Saga', 'AQ Zion Saga')
('AQ Mforce Gen X', 'AQ Mforce Gen X')


In [27]:
con.commit()

Now we can use those shorter display names for our charts.

[Back to contents](#contents)

---

### Missing values

There is only one row with missing values in the whole Database. Let's print that row.

In [28]:
query='''
SELECT *
FROM fact_sales_monthly
WHERE fiscal_year IS NULL
'''

cursor.execute(query)
print(*cursor.fetchall(), sep='\n')

It's for product `A0` during `June 2019`. It could mean that this product didn't get any sales that month. Let's see some more info about it.

In [29]:
# Look for other sales of this product
query='''
SELECT *
FROM fact_sales_monthly
WHERE product_code = "A0"
'''

cursor.execute(query)
print(*cursor.fetchall(), sep='\n')

There are no other record of sales of this product.

In [30]:
# What product is this
query='''
SELECT *
FROM dim_product
WHERE product_code = "A0"
'''

cursor.execute(query)
print(*cursor.fetchall(), sep='\n')

There is no register of this product in `dim_product`. It doesn't exist. We can delete this row.

In [31]:
# What product is this
query='''
DELETE 
FROM fact_sales_monthly
WHERE product_code = "A0"
'''

cursor.execute(query)
print(*cursor.fetchall(), sep='\n')

con.commit()

[Back to Contents](#contents)

---

### Duplicates

The tables don't have Primary keys set up. That means it's possible that some of them have duplicated values in critical columns.

We'll be examining various column combinations in each table. Unless we encounter a non-zero value, we'll proceed to the next one.

In [32]:
# No duplicate customer codes
table = 'dim_customer'
columns = ['customer_code']
count_duplicates(table, columns);

Table: dim_customer
Duplicates found: 0


In [33]:
# Customer codes are unique to the ['customer', 'platform', 'channel', 'region'] combination
table = 'dim_customer'
columns = ['customer', 'platform', 'channel', 'region']
count_duplicates(table, columns);

Table: dim_customer
Duplicates found: 101


We got duplicates here. It's possible that the batabase notes are mistaken and `customer_code` is given by market, instead of by region.

In [34]:
# Check if adding 'market' to the previous query makes the results unique
table = 'dim_customer'
columns = ['customer', 'platform', 'channel', 'region', 'sub_zone', 'market']
count_duplicates(table, columns);

Table: dim_customer
Duplicates found: 0


It seems to be the case. We can move on.

In [35]:
# No duplicate product_code
table = 'dim_product'
columns = ['product_code']
count_duplicates(table, columns);

Table: dim_product
Duplicates found: 0


In [36]:
# No duplicate variants for the same product
table = 'dim_product'
columns = ['product', 'variant']
count_duplicates(table, columns);

Table: dim_product
Duplicates found: 0


In [37]:
# No duplicate category for the same product variant
table = 'dim_product'
columns = ['category', 'product', 'variant']
count_duplicates(table, columns);

Table: dim_product
Duplicates found: 0


In [38]:
# No duplicate segment for the same category product variant
table = 'dim_product'
columns = ['segment', 'category', 'product', 'variant']
count_duplicates(table, columns);

Table: dim_product
Duplicates found: 0


In [39]:
# No duplicate division for the same category product variant
table = 'dim_product'
columns = ['division', 'segment', 'category', 'product', 'variant']
count_duplicates(table, columns);

Table: dim_product
Duplicates found: 0


In [40]:
# The combination ['product', 'category', 'variant', 'segment', 'division'] should be unique
table = 'dim_product'
columns = ['product', 'category', 'variant', 'segment', 'division']
count_duplicates(table, columns);

Table: dim_product
Duplicates found: 0


In [41]:
# Each customer_code should have only one discount per fiscal_year
table = 'fact_pre_discount'
columns = ['customer_code', 'fiscal_year']
count_duplicates(table, columns);

Table: fact_pre_discount
Duplicates found: 0


In [42]:
# Each product_code should have only one cost per year
table = 'fact_manufacturing_cost'
columns = ['product_code', 'cost_year']
count_duplicates(table, columns);

Table: fact_manufacturing_cost
Duplicates found: 0


In [43]:
# Each product_code should have only one price per year
table = 'fact_gross_price'
columns = ['product_code', 'fiscal_year']
count_duplicates(table, columns);

Table: fact_gross_price
Duplicates found: 0


In [44]:
# The sales data should be aggrergated by customer, product and date.
table = 'fact_sales_monthly'
columns = ['date', 'product_code', 'customer_code']
count_duplicates(table, columns);

Table: fact_sales_monthly
Duplicates found: 0


We only found duplicates in `dim_customer`, and they are easily explainable. There is nothing to fix here.

[Back to Contents](#contents)

---

### Other

We should check the consistency of the data that we have, and that it obeys the rules stated in the Database documentation.

Let's see if the time period is the same in all tables.

In [45]:
# Period for fact_pre_discount data
query = """
SELECT MIN(fiscal_year), MAX(fiscal_year)
FROM fact_pre_discount
"""

cursor.execute(query)
cursor.fetchone()

(2018, 2022)

In [46]:
# Period for fact_manufacturing_cost data
query = """
SELECT MIN(cost_year), MAX(cost_year)
FROM fact_manufacturing_cost
"""

cursor.execute(query)
cursor.fetchone()

(2018, 2022)

In [47]:
# Period for fact_gross_price data
query = """
SELECT MIN(fiscal_year), MAX(fiscal_year)
FROM fact_gross_price
"""

cursor.execute(query)
cursor.fetchone()

(2018, 2022)

In [48]:
# Period for fact_sales_monthly data
query = """
SELECT MIN(fiscal_year), MAX(fiscal_year)
FROM fact_sales_monthly
"""

cursor.execute(query)
cursor.fetchone()

(2018, 2022)

All data is from fiscal years `2018` to `2022`

---

Some columns are supposed to have only certain values. Let's make sure that's the case, and there are no errors or typos.

In [49]:
# dim_customer.platform should have 2 values
query = """
SELECT DISTINCT platform
FROM dim_customer
"""
cursor.execute(query)
print(*cursor.fetchall(), sep="\n")

('Brick & Mortar',)
('E-Commerce',)


In [50]:
# dim_customer.channel should have 3 values
query = """
SELECT DISTINCT channel
FROM dim_customer
"""
cursor.execute(query)
print(*cursor.fetchall(), sep="\n")

('Direct',)
('Distributor',)
('Retailer',)


In [51]:
# dim_product.division should have 3 values
query = """
SELECT DISTINCT division
FROM dim_product
"""
cursor.execute(query)
print(*cursor.fetchall(), sep="\n")

('P & A',)
('PC',)
('N & S',)


In [52]:
# dim_product.segment should have 6 values
query = """
SELECT DISTINCT segment
FROM dim_product
"""
cursor.execute(query)
print(*cursor.fetchall(), sep="\n")

('Peripherals',)
('Accessories',)
('Notebook',)
('Desktop',)
('Storage',)
('Networking',)


---

There are some relationships that should be constant. For example, each market belongs to exactly one sub_zone. Let's check those relationships next.

The following cells should output `1` if there is no errors.

In [53]:
# Each dim_customer.customer_code should have 1 platform
query = """
SELECT customer_code, count(platform) as platform_count
FROM dim_customer
GROUP BY customer_code
ORDER BY platform_count DESC
"""
cursor.execute(query)
print (f'Max platform count: {cursor.fetchone()[1]}')

Max platform count: 1


In [54]:
# Each dim_customer.market should have 1 sub-zone
query = """
SELECT market, count(sub_zone) as sub_zone_count
FROM (
    SELECT DISTINCT market, sub_zone
    FROM dim_customer
)
GROUP BY market
ORDER BY sub_zone_count DESC
"""
cursor.execute(query)
print (f'Max sub zone count: {cursor.fetchone()[1]}')

Max sub zone count: 1


In [55]:
# Each dim_customer.market should have 1 region
query = """
SELECT market, count(region) as region_count
FROM (
    SELECT DISTINCT market, region
    FROM dim_customer
)
GROUP BY market
ORDER BY region_count DESC
"""
cursor.execute(query)
print (f'Max region count: {cursor.fetchone()[1]}')

Max region count: 1


---

Some columns should stay within reasonable values. Let's check that next.

In [56]:
# fact_pre_discount.pre_invoice_discount_pct should always be less than 1
query = """
SELECT max(pre_invoice_discount_pct)
FROM fact_pre_discount
"""
cursor.execute(query)
print(*cursor.fetchall(), sep="\n")

(0.3099,)


In [57]:
# fact_sales_monthly.date consistently assigns the same months to the right fiscal_year
query= """
SELECT fiscal_year, MIN(date), MAX(date)
FROM fact_sales_monthly
GROUP BY fiscal_year
"""
cursor.execute(query)
print(*cursor.fetchall(), sep="\n")

(2018, '2017-09-01', '2018-08-01')
(2019, '2018-09-01', '2019-08-01')
(2020, '2019-09-01', '2020-08-01')
(2021, '2020-09-01', '2021-08-01')
(2022, '2021-09-01', '2021-12-01')


There is only 3 months worth of data for fiscal year 2022, as opposed to the full 12 months for the rest of the fiscal years.

In [58]:
# fact_sales_monthly.sold_quantity should always be positive
query= """
SELECT MIN(sold_quantity)
FROM fact_sales_monthly
"""
cursor.execute(query)
print(*cursor.fetchall(), sep="\n")

(1,)


There are some items with `sold_quantity` of `0`. It's possible that means that the item didn't get any sales that month, but unlikely. 

Let's see those sales.

In [59]:
query= """
SELECT DISTINCT date, product_code, sold_quantity
FROM fact_sales_monthly
WHERE sold_quantity = 0
"""
cursor.execute(query)
print(*cursor.fetchall(), sep="\n")

All the sales with 0 quantity are from Sept 2017.

In [60]:
query = """
SELECT DISTINCT product
FROM (
    SELECT *
    FROM fact_sales_monthly
    WHERE sold_quantity = 0
) AS zero_sales 
INNER JOIN dim_product 
ON zero_sales.product_code = dim_product.product_code
"""
cursor.execute(query)
print(*cursor.fetchall(), sep='\n')

Only two products have this issue. We can delete these rows.

In [61]:
query = """
DELETE
FROM fact_sales_monthly
WHERE sold_quantity = 0
"""
cursor.execute(query)
con.commit()

The data is ready for analysis.

[Back to contents](#contents)

---

## Analysis

### Finding the bestsellers

There are different approaches that we can take here. We can find sales by month to look for seasonality, by year to look at trends, or historical to see products that perform strongly year after year.

We can look at units sold, revenue, or profit. There's also the option of ranking by total values, or by their contribution (ratio) towards the grand total.

Right now we will only look at two of them, and leave the rest for the Dashboard.

The ones that we'll analyze here are:
- Most units sold in the last full year (2021)
- Most profit generated per year

We only have access to sales data of 4 of it's products. Lets see what they are.

In [62]:
pd.read_sql('SELECT * from dim_product LIMIT 1', con)

,product_code,division,segment,category,product,variant,display_name
0,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,AQ Dracula HDD


In [63]:
query="""
SELECT display_name, product, division, segment, category
FROM dim_product
    JOIN fact_sales_monthly USING(product_code)
GROUP BY product
"""

pd.read_sql(query, con)

,display_name,product,division,segment,category
0,AQ Dracula HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,P & A,Peripherals,Internal HDD
1,AQ Mforce Gen X,AQ Mforce Gen X,P & A,Peripherals,Graphic Card
2,AQ Werewolf NAS HDD,AQ WereWolf NAS Internal Hard Drive HDD – 8.89 cm,P & A,Peripherals,Internal HDD
3,AQ Zion Saga,AQ Zion Saga,P & A,Peripherals,Internal HDD


There isn't more details about the `Zion Saga`, so we'll assume it is a general use HDD, as the Dracula HDD.

So there is two general HDD, one NAS HDD, and a graphic card.

We'll begin by finding the most units sold.

In [64]:
query = """
SELECT fiscal_year, dim_product.product_code, display_name, variant, SUM(sold_quantity) AS units_sold
FROM fact_sales_monthly 
    JOIN dim_product USING(product_code)
WHERE fiscal_year = 2021
GROUP BY fiscal_year, dim_product.product_code
ORDER BY units_sold DESC
"""

best_sell_df = pd.read_sql_query(query, con)
best_sell_df.head(5)


,fiscal_year,product_code,display_name,variant,units_sold
0,2021,A0219150201,AQ Werewolf NAS HDD,Standard,100799
1,2021,A0118150101,AQ Dracula HDD,Standard,100039
2,2021,A0321150302,AQ Zion Saga,Plus,99955
3,2021,A0118150102,AQ Dracula HDD,Plus,98124
4,2021,A0118150104,AQ Dracula HDD,Premium Plus,97931


In [65]:

code_to_variant = dict(zip(best_sell_df['product_code'], best_sell_df['variant']))

fig = px.bar(data_frame=best_sell_df.sort_values(by='units_sold', ascending=False),
             x='product_code',
             y='units_sold',
             color='display_name',
             labels={
                 'product_code': 'Product variants',
                 'units_sold': 'Units sold'
                 },
             title='2021 Sales by Product Variant',
             )

fig.update_xaxes(labelalias =  code_to_variant)
fig.update_layout(legend_title_text='Product')
fig.show()

Most product variants had similar sales last year. The Dracula HDD was the best selling product overall (4 variants vs 3 of the Werewolf NAS and Zion Saga).

The Mforce Gen X is lagging behind.

How about yearly profit?

In [66]:
# Yearly profit
# First we'll create a view with the profit per sale.

query = """
CREATE VIEW IF NOT EXISTS profit_per_sale AS 
SELECT *, CAST((total_revenue - total_cost) AS int) AS total_profit
FROM (
    SELECT 
        fact_sales_monthly.*,
        gross_price,
        manufacturing_cost,
        pre_invoice_discount_pct,
        ((gross_price * (1 - pre_invoice_discount_pct)) * sold_quantity) AS total_revenue,
        (manufacturing_cost * sold_quantity) AS total_cost
        
    FROM fact_sales_monthly
        JOIN fact_manufacturing_cost 
            ON fact_manufacturing_cost.product_code = fact_sales_monthly.product_code
            AND fact_manufacturing_cost.cost_year = fact_sales_monthly.fiscal_year
        JOIN fact_gross_price 
            ON fact_gross_price.product_code = fact_sales_monthly.product_code
            AND fact_gross_price.fiscal_year = fact_sales_monthly.fiscal_year
        JOIN fact_pre_discount 
            ON fact_pre_discount.customer_code = fact_sales_monthly.customer_code
            AND fact_pre_discount.fiscal_year = fact_sales_monthly.fiscal_year
)

"""
cursor.execute(query);

In [67]:
# Calculate the profit per year

query = """
SELECT dim_product.product_code, display_name, variant, profit_per_sale.fiscal_year, SUM(total_profit) AS profit
FROM dim_product 
    JOIN profit_per_sale USING(product_code)
GROUP BY profit_per_sale.product_code, profit_per_sale.fiscal_year
ORDER BY profit DESC
"""

yearly_profit_df = pd.read_sql_query(query, con)
yearly_profit_df.head(5)

,product_code,display_name,variant,fiscal_year,profit
0,A0321150302,AQ Zion Saga,Plus,2022,2383543
1,A0321150303,AQ Zion Saga,Premium,2022,2227494
2,A0418150103,AQ Mforce Gen X,Standard 3,2022,2201061
3,A0219150202,AQ Werewolf NAS HDD,Plus,2022,2032675
4,A0220150203,AQ Werewolf NAS HDD,Premium,2022,2031093


In [68]:
# Create the stacked bar chart
fig = px.bar(yearly_profit_df.sort_values(by='fiscal_year', ascending=True),
              x='display_name',
              y='profit',
              color='variant',
              facet_col='fiscal_year',
              barmode='stack',
              facet_col_wrap=5,
              )

# Update axis titles
fig.update_xaxes(title_text='', showticklabels=True)
fig.update_yaxes(title_text='Profit')


annot = []
offset = 0.2
x_pos = 0.1
for year in yearly_profit_df.sort_values(by='fiscal_year', ascending=True)['fiscal_year'].unique():
    annot.append(
        dict(
            text = f'{year}',
            x = x_pos,
            y = 1,
            showarrow = False,
            xref = 'paper',
            yref='paper'
        )
    )
    x_pos += offset

# Update facet titles
fig.update_layout(
    title='Yearly Profit by Product',
    showlegend=True,
    legend_title='Variants',
    yaxis=dict(title=''), 
    annotations=annot
)

fig.show()

There has been a remarkable grow in the last years. Remember that this data has only the first 3 months of 2022!

The Mforce Gen X is also struggling to keep up when measuring profit. 

[Back to Contents](#contents)

---

### Popularity across time and markets

How has popularity changed over time and across markets?

In this section we'll be looking for seasonality, and patterns in markets.

First, lets see how each product sells during each month. Since we only have 3 months on 2022 we'll exclude it from this chart. We only want full years.

In [69]:
# Product sales per month
query = """
SELECT 
    display_name,
    date, 
    CAST(fiscal_year AS text) as fiscal_year, 
    SUBSTR('JanFebMarAprMayJunJulAugSepOctNovDec', 3 * CAST(strftime('%m', date) AS int) - 2, 3) AS month, 
    SUM(sold_quantity) as total_sold
FROM fact_sales_monthly
    JOIN dim_product USING(product_code)
WHERE fiscal_year != 2022
GROUP BY product, date
ORDER BY product, date ASC
"""

sales_by_month_df = pd.read_sql_query(query, con)


In [70]:
# We'll use this color palette for the next few charts
custom_colors=['#003f5c','#58508d', '#bc5090', '#ff6361', '#ffa600']

In [71]:
# Build a stacked bar chart with sales on each year.
fig = px.bar(data_frame=sales_by_month_df,
             x='month',
             y='total_sold',
             facet_col='display_name',
             color='fiscal_year',
             facet_col_wrap=1,
             facet_row_spacing=0.12,
             width=800,
             height=800,
             color_discrete_sequence=custom_colors
             )

# Update axis titles
fig.update_xaxes(title_text='', showticklabels=True)
fig.update_yaxes(title_text='')

# Remove "product=" prefix from facet labels
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.update_layout(
    title='Total sales by month',
    legend_title_text='Fiscal year',
    showlegend=True
)


fig.show()

Notice that the chart starts in September. That is the start of Atliq's fiscal year.

There is a jump in sales in the last months of the year. It happens with all the products.

It shows very clearly if we plot a historic chart.

In [72]:
# Build a stacked bar chart with sales on each year.

fig = px.bar(data_frame=sales_by_month_df,
             x='date',
             y='total_sold',
             facet_col='display_name',
             facet_col_wrap=1,
             color='fiscal_year',
             width =800,
             height=600,
             facet_row_spacing=0.12,

             color_discrete_sequence=custom_colors)

fig.update_yaxes(title='')
fig.update_xaxes(title='')

fig.for_each_annotation(lambda a: a.update(text = a.text.split('=')[1]))

fig.update_layout(
    title='Historic sales by month',
    legend_title_text='Fiscal year'
)
fig.show()

When it comes to popularity, three of our four products are growing year after year. These are:  Dracula HDD, WereWolf NAS and Zion Saga.

On the other hand, the Mforce Gen X has been dropping, and it appears that 2021 was it's weakest year.

---

Now lets analyze their performance in different markets.

First let's see which are our best performing markets.

In [73]:
query = """
SELECT market, MIN(fiscal_year) AS active_since, SUM(sold_quantity) AS units_sold
FROM fact_sales_monthly
    JOIN dim_customer USING(customer_code)
GROUP BY market
ORDER BY units_sold DESC
"""

market_sales_df = pd.read_sql_query(query,con)
market_sales_df.head(5)

,market,active_since,units_sold
0,India,2018,1087710
1,USA,2018,609825
2,South Korea,2018,341767
3,Canada,2018,197068
4,Philiphines,2018,191884


Now lets see how our products perform in these markets.


In [74]:
top_markets = ', '.join(market_sales_df.head(5)['market'].apply(lambda x: f"'{x}'"))

In [75]:
# yearly sales for a line chart
query = f"""
SELECT fiscal_year, market, display_name, SUM(sold_quantity) AS sales
FROM fact_sales_monthly
    JOIN dim_product USING(product_code)
    JOIN dim_customer USING(customer_code)
GROUP BY fiscal_year, market, product
HAVING market IN ({top_markets})
"""

top_market_totals = pd.read_sql_query(query, con)

In [76]:
# Plot the sales line chart
fig = px.line(top_market_totals,
            x='fiscal_year',
            y='sales',
            line_group='market',
            facet_col='display_name',
            color='market',
            facet_col_spacing=0.04
            )

# Set the title and shorten the tick labels
fig.update_xaxes(title='Year',
                 tickvals=[2018, 2019, 2020, 2021, 2022],
                 ticktext=["'18", "'19", "'20", "'21", "'22"])

# Remove the 'market=' part of the carets labels
fig.for_each_annotation(lambda a: a.update(text = a.text.split('=')[1]))


# Set y-axis title only for the outer facet (anchor == 'x')
# If we simply do:
#                   fig.update_yaxes(title='Units sold')
# Then the title will show up twice.
fig.for_each_yaxis(lambda y: y.update(title='Units sold') if y.anchor == 'x' else y.update(title=''))

fig.update_layout(
    title='Product sales',
    legend_title_text='Market'
)

fig.show()

The Dracula HDD is the only product that on 2022 still hasn't cought up with the sales of the previous years, but since we only have 3 months of data for 2022, it is likely that the final sales will outperform every other year 

The Mforce Gen X saw a marked decline on 2021. It is recovering on 2022, and already surpassing the sales of previous years.

The Werewolf NAS and Zion Saga have seen continious growth since they were introduced.

[Back to Contents](#contents)

---

### Variant sales

Each one of our products have several variants. Is there any variant with a disproportionate contribution the to product total sales? 

To answer this question, lets start by taking a look at the total monthly sales of each variant, and their contribution to the monthly total.

First we'll create a view that contains the monthly info on sales, cost and price.

In [77]:
# Create the VIEW with info on the monthly sales

query="""
CREATE VIEW IF NOT EXISTS monthly_sales_full AS
SELECT 
    dim_product.display_name, 
    dim_product.variant,
    fact_sales_monthly.product_code, 
    fact_sales_monthly.date,
    fact_sales_monthly.fiscal_year,
    SUM(fact_sales_monthly.sold_quantity) as variant_sales,
    fact_gross_price.gross_price,
    fact_manufacturing_cost.manufacturing_cost,
    monthly_product_sales.product_sales,
    CAST(SUM(fact_sales_monthly.sold_quantity) as REAL) / monthly_product_sales.product_sales AS variant_ratio
FROM
    fact_sales_monthly
    JOIN dim_product USING(product_code)
    JOIN fact_gross_price USING(product_code, fiscal_year)
    JOIN fact_manufacturing_cost ON 
        fact_sales_monthly.product_code = fact_manufacturing_cost.product_code
        AND fact_sales_monthly.fiscal_year = fact_manufacturing_cost.cost_year
    JOIN (
        SELECT display_name, date, SUM(sold_quantity) AS product_sales
        FROM fact_sales_monthly
            JOIN dim_product USING(product_code)
        GROUP BY 
            display_name,
            date
        ) AS monthly_product_sales
        USING(display_name, date)
GROUP BY 
    fact_sales_monthly.product_code, 
    fact_sales_monthly.date,
    fact_sales_monthly.fiscal_year
"""

cursor.execute(query);

Now we can check if there are some extreme values for ratios.

In [78]:
query="""
SELECT MIN(variant_ratio), MAX(variant_ratio), AVG(variant_ratio)
FROM monthly_sales_full
"""

pd.read_sql(query, con)

,MIN(variant_ratio),MAX(variant_ratio),AVG(variant_ratio)
0,0.183456,1.0,0.33463


The MIN being `0.18` tells us that that the variants are contributing a significant amount to their product sales.

The MAX being `1` means that at some point, some products had only one variant available for sale.

In [79]:
query="""
SELECT DISTINCT display_name, MIN(date), MAX(date)
FROM monthly_sales_full
WHERE variant_ratio = 1
GROUP BY display_name
"""

pd.read_sql(query,con)

,display_name,MIN(date),MAX(date)
0,AQ Mforce Gen X,2020-09-01,2021-12-01
1,AQ Zion Saga,2019-09-01,2020-08-01


The products with no variants were:
- AQ Zion Saga for it's first year in the market. After that more variants were introduced.
- AQ Mforce Gen X after a few years. It makes sense to simplify the offer on an ageing graphic card.

In [80]:
# Dataframe for the chart
variant_sales_df = pd.read_sql("SELECT display_name, variant, date, variant_sales FROM monthly_sales_full", con)


# Build a stacked area chart with the monthly sales per product.

fig = px.area(data_frame=variant_sales_df,
             x='date',
             y='variant_sales',
             facet_col='display_name',
             facet_col_wrap=1,
             color='variant',
             facet_row_spacing=0.05,
             width=1000,
             height=1200,
             )

# Set labels
fig.update_xaxes(title='')
fig.update_yaxes(title='Units sold')

# remove the 'display_name=' part of the label
fig.for_each_annotation(lambda a: a.update(text = a.text.split('=')[1]))

# Show x-axis tick labels on all facets
for axis in fig.layout:
    if axis.startswith('xaxis'):
        fig.layout[axis]['showticklabels'] = True

fig.update_layout(
    title='Historic variant sales',
    legend_title_text='Variants'
)

fig.show()

This shows us that all the variants that are introduced have a significant contribution to the total sales. None of the variants being currently offered is irrelevant.

[Back to Contents](#contents)

---

### Channel sales

To understand the distribution of sales across different customer channels, we will analyze the contribution of each channel to the sales of our products. This will help identify if certain channels drive the demand of some of our products.

In [81]:
# calculate channel sales per product

query = """
SELECT display_name, channel, SUM(sales_by_channel.channel_sales) as channel_sales, SUM(total_sales.variant_sales) as total_sales
FROM (
    SELECT display_name, variant, product_code, channel, SUM(sold_quantity) as channel_sales
    FROM fact_sales_monthly
        JOIN dim_customer USING(customer_code)
        JOIN dim_product USING(product_code)
    GROUP BY fact_sales_monthly.product_code, channel) as sales_by_channel
    JOIN (
        SELECT product_code, SUM(sold_quantity) as variant_sales
        FROM fact_sales_monthly
        GROUP BY product_code) as total_sales USING(product_code)
GROUP BY display_name, channel
"""

channel_sales_df = pd.read_sql_query(query, con)

In [82]:
# Plot a stacked bar chart

fig = px.bar(channel_sales_df,
             x='display_name',
             y='channel_sales',
             color='channel',
             barmode='stack',
             title='Sales by channel'
)

# Fix titles
fig.update_yaxes(title='Units sold')
fig.update_xaxes(title='')
fig.update_legends(title='Channel')


fig.show()

According to the total sales, retailers are the main sales channel for these products. 

Let's see if that is true in an historical chart.

In [83]:
# calculate MONTHLY channel sales per product 

query = """
SELECT display_name, channel, date, SUM(sales_by_channel.channel_sales) as channel_sales
FROM (
    SELECT display_name, date, variant, product_code, channel, SUM(sold_quantity) as channel_sales
    FROM fact_sales_monthly
        JOIN dim_customer USING(customer_code)
        JOIN dim_product USING(product_code)
    GROUP BY fact_sales_monthly.product_code, channel, date) as sales_by_channel

GROUP BY display_name, channel, date
"""
"""
    JOIN (
        SELECT product_code, SUM(sold_quantity) as variant_sales
        FROM fact_sales_monthly
        GROUP BY product_code) as total_sales USING(product_code)
"""

monthly_channel_sales_df = pd.read_sql_query(query, con)

In [84]:
# Plot historic bar chart

fig=px.bar(monthly_channel_sales_df,
           x='date',
           y='channel_sales',
           color='channel',
           barmode='stack',
           facet_col='display_name',
           facet_col_wrap=1,
           height=800,
           title='Retailers are the main demand drivers')

# Remove the 'display_name=' part of the annotations
fig.for_each_annotation(lambda a: a.update(text = a.text.split('=')[1]))

fig.update_yaxes(title='Units sold')

fig.show()

The `Direct` and `Distributor` channels show an activity increase on the last fiscal year (Sept 2021). Before that, almost all the demand was from `Retailer`, and is still the predominant channel.

[Back to Contents](#contents)

---

### Product margins

What are the products with the best and worst margins? 

So far we mostly analyzed sales by units. Now it's time to see what kind of margins do they leave.

In [95]:
# Create a view with profit margin data
query = """
CREATE VIEW IF NOT EXISTS product_margins as
SELECT 
    prod.product_code,
    prod.product,
    prod.variant,
    price.fiscal_year,
    cost.manufacturing_cost,
    price.gross_price,
    (price.gross_price - cost.manufacturing_cost) as margin,
    (price.gross_price - cost.manufacturing_cost) / price.gross_price as margin_rate
FROM
    dim_product as prod
    JOIN fact_manufacturing_cost as cost USING(product_code)
    JOIN fact_gross_price as price ON prod.product_code = price.product_code AND cost.cost_year = price.fiscal_year
"""

cursor.execute(query);

Now let's check the best and worst margins.

In [97]:
# Looking for the best and worst margins
query="""
SELECT MAX(margin), MIN(margin), MAX(margin_rate), MIN(margin_rate)
FROM product_margins
"""

pd.read_sql_query(query, con)

,MAX(margin),MIN(margin),MAX(margin_rate),MIN(margin_rate)
0,630.6054,1.9791,0.714225,0.687511


Based on rates, the margins have a very narrow range around `0.7`.

The flat margins are much different, however since the margin is almost constant, that only tells us that the company has a wide array of products with diferent price ranges.

In [120]:
# Best flat margin

query="""
SELECT product, variant, fiscal_year, margin
FROM product_margins
ORDER BY margin DESC
LIMIT 5
"""

pd.read_sql_query(query, con)

,product,variant,fiscal_year,margin
0,AQ HOME Allin1 Gen 2,Plus 2,2022,630.6054
1,AQ HOME Allin1 Gen 2,Plus 3,2022,625.1927
2,AQ HOME Allin1 Gen 2,Standard 1,2022,623.9115
3,AQ BZ Allin1 Gen 2,Premium 2,2022,612.9470
4,AQ BZ Allin1 Gen 2,Plus 2,2022,608.2999


In [119]:
# Worst flat margin

query="""
SELECT product, variant, fiscal_year, margin
FROM product_margins
ORDER BY margin
LIMIT 5
"""

pd.read_sql_query(query, con)

,product,variant,fiscal_year,margin
0,AQ Master wired x1 Ms,Standard 1,2019,1.9791
1,AQ Master wired x1 Ms,Standard 1,2021,2.0239
2,AQ Master wired x1 Ms,Standard 1,2020,2.0248
3,AQ Master wired x1 Ms,Standard 1,2018,2.0555
4,AQ Pen Drive DRC,Standard,2020,2.0680


In [121]:
# Best margin rate

query="""
SELECT product, variant, fiscal_year, margin_rate
FROM product_margins
ORDER BY margin_rate DESC
LIMIT 5
"""

pd.read_sql_query(query, con)

,product,variant,fiscal_year,margin_rate
0,AQ Wi Power Dx1,Premium,2020,0.714225
1,AQ Mforce Gen Y,Premium 1,2021,0.714204
2,AQ GEN Z,Standard Red,2022,0.714203
3,AQ Master wired x1,Plus 2,2018,0.714185
4,AQ Smash 1,Standard Firey Red,2021,0.714167


In [122]:
# Worst margin rate

query="""
SELECT product, variant, fiscal_year, margin_rate
FROM product_margins
ORDER BY margin_rate
LIMIT 5
"""

pd.read_sql_query(query, con)

,product,variant,fiscal_year,margin_rate
0,AQ Digit SSD,Premium,2020,0.687511
1,AQ Master wired x1 Ms,Standard 1,2021,0.687606
2,AQ Master wireless x1,Premium 2,2021,0.687636
3,AQ Gamers,Plus 2,2020,0.687637
4,AQ Aspiron,Plus Red,2022,0.687677


There are no products generating losses. 

All products are maintaining a consistent profit margin of 0.7 ± 0.015, indicating that prices are effectively tracking production costs to preserve this margin and prevent losses.

[Back to Contents](#contents)

---

---

HYPOTHESIS: The Dracula HDD sales on 2022 will outperform every other year. 

In [123]:
query="""
SELECT product, fiscal_year, margin_rate
FROM product_margins
"""

margins_df = pd.read_sql_query(query, con)

In [124]:


fig = px.scatter(margins_df,
                 x='fiscal_year',
                 y='margin_rate')

fig.show()